In [15]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

# Simulação de série temporal
np.random.seed(42)
N = 1000
df = pd.DataFrame({'Y': np.random.randn(N).cumsum()})

# Indicador: média móvel
df['X1'] = df['Y'].rolling(window=3).mean().shift(1)

# Lags
df['Y_lag1'] = df['Y'].shift(1)
df['Y_lag2'] = df['Y'].shift(2)

df.dropna(inplace=True)

# Split em treino + último registro para previsão futura
train = df.iloc[:-1]
last_row = df.iloc[-1:]

X_train = train[['X1', 'Y_lag1', 'Y_lag2']]
y_train = train['Y']

# Treinamento
model = XGBRegressor()
model.fit(X_train, y_train)

df.head()
pd.concat([X_train, y_train], axis=1)

,X1,Y_lag1,Y_lag2,Y
3,0.620434,1.006138,0.358450,2.529168
4,1.297919,2.529168,1.006138,2.295015
5,1.943441,2.295015,2.529168,2.060878
6,2.295020,2.060878,2.295015,3.640091
7,2.665328,3.640091,2.060878,4.407525
...,...,...,...,...
994,18.711892,18.174843,18.856827,17.173223
995,18.068298,17.173223,18.174843,16.892123
996,17.413396,16.892123,17.173223,18.689809
997,17.585052,18.689809,16.892123,19.330652


In [16]:
# Forecast 3 steps ahead (iterativo):
forecast_steps = 3
predictions = []

current_Y_lag1 = last_row['Y'].values[0]
current_Y_lag2 = last_row['Y_lag1'].values[0]
current_X1 = last_row['X1'].values[0]

for step in range(forecast_steps):
    # Monta X para próximo passo
    X_next = pd.DataFrame({
        'X1': [current_X1],
        'Y_lag1': [current_Y_lag1],
        'Y_lag2': [current_Y_lag2]
    })
    display(X_next)
    
    # Faz previsão
    y_pred = model.predict(X_next)[0]
    predictions.append(y_pred)
    
    # Atualiza lags para próxima iteração
    current_Y_lag2 = current_Y_lag1
    current_Y_lag1 = y_pred
    current_X1 = np.mean([y_pred, current_Y_lag1, current_Y_lag2])  # Exemplo simples de atualização de média móvel "na mão"

print(predictions)


,X1,Y_lag1,Y_lag2
0,18.926645,19.332056,18.759473


,X1,Y_lag1,Y_lag2
0,18.725618,18.4224,19.332056


,X1,Y_lag1,Y_lag2
0,19.543022,20.103334,18.4224


[18.4224, 20.103334, 20.534325]
